## Import packages

In [12]:
# Set auto reload for python functions
%reload_ext autoreload
%autoreload 2

# Import packages
import glob
import tqdm
import numpy as np
from time import time
import tensorflow as tf
from M_matrix import _parse_function
from M_matrix import gen_reduce_max_tensors
from keras.applications.inception_v3 import preprocess_input

# GoogLeNet # Need to install lucid with the command: `pip install lucid`
import lucid.modelzoo.vision_models as models
import lucid.optvis.render as render

## Set basic info

In [17]:
# Layer info
layer_sizes = {
    'mixed3a': 256,
    'mixed3b': 480,
    'mixed4a': 508,
    'mixed4b': 512,
    'mixed4c': 512,
    'mixed4d': 528,
    'mixed4e': 832,
    'mixed5a': 832,
    'mixed5b': 1024
}
layers = list(layer_sizes.keys())


# Hyperparameters
num_of_classes = 1000
prob_mass_threshold = 0.1
batch = 200

# Input directory path
input_dir_path = '/Users/haekyu/data/imagenet-tf-records'

# Output directory path
output_dir_path = '/Users/haekyu/data'

## Import googlenet model

In [9]:
googlenet = models.InceptionV1()
googlenet.load_graphdef()

## Get A-matrix

### Initialize A-matrices

(Still editing, copy and paste summit paper, section 6.1) We want to understand what a neural network is detecting in a dataset. We propose summarizing how an image dataset is represented through-out a CNN by aggregating individual image activations at each channel in the network, over all of the images in a given class. This aggregation results in a matrix, Al for each layer l in a network, where an entry Al cj roughly represents how important channel j (from the lth layer) is for representing images from class c.

We define As to keep the aggregation result.
```
As: a dictionary, where
    - key: layer name in string (e.g., "mixed3a")
    - val: a matrix (np.array(dtype=int)) of shape (#class, #channels)
```

In [10]:
As = {}
for layer in layer_sizes:
    num_of_channels_in_layer = layer_sizes[layer]
    A = np.zeros([num_of_classes, num_of_channels_in_layer], dtype=int)
    As[layer] = A

### Aggregate the activation and complete A-matrices

In [14]:
# Time checker
start_time = time()

# Get file paths
file_paths = glob.glob('{}/*'.format(input_dir_path))

# Get A-matrix
with tf.Graph().as_default():
    
    # Define parsed dataset tensor
    dataset = tf.data.TFRecordDataset(file_paths)
    dataset = dataset.map(_parse_function)
    dataset = dataset.map(lambda img, lab, syn: (preprocess_input(img), lab, syn))
    dataset = dataset.batch(batch)

    # Define iterator through the datasets
    iterator = dataset.make_one_shot_iterator()
    t_preprocessed_images, t_labels, t_synsets = iterator.get_next()
    
    # Define actiavtion map render
    T = render.import_model(googlenet, t_preprocessed_images, None)
    t_act_max_all_layers = gen_reduce_max_tensors(layers, T)
    
    # Read data
    progress_counter, image_id = 0, -1
    with tf.Session() as sess:
        
        tic = time()

        try:
            with tqdm.tqdm(total=1243025, unit='imgs') as pbar:
                while(True):
                    
                    # Get parsed data
                    progress_counter += 1
                    act_maxs_3a, act_maxs_3b, act_maxs_4a, act_maxs_4b, \
                    act_maxs_4c, act_maxs_4d, act_maxs_4e, act_maxs_5a, \
                    act_maxs_5b, labels, synsets = \
                        sess.run([*t_act_max_all_layers, t_labels, t_synsets])
                    
                    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
                    no sess.run after this!
                    python code here on out
                    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
                    
                    # Define a list for activation maximums of all layers
                    act_maxs_all_layers = {
                        'mixed3a': act_maxs_3a,
                        'mixed3b': act_maxs_3b,
                        'mixed4a': act_maxs_4a,
                        'mixed4b': act_maxs_4b,
                        'mixed4c': act_maxs_4c,
                        'mixed4d': act_maxs_4d,
                        'mixed4e': act_maxs_4e,
                        'mixed5a': act_maxs_5a,
                        'mixed5b': act_maxs_5b
                    }
                    
                    # Get the current batch size.
                    # It could be smaller than batch in tail
                    curr_batch_size = labels.shape[0] 
                    
                    # For an image i
                    for i in range(curr_batch_size):
                        
                        # Image id
                        image_id += 1
                        
                        # Initialize the co-activated neurons info
                        co_activated_neurons[image_id] = {'label': labels[i] - 1}

                        # For each layer's activation maximums
                        for layer in layers:
                            
                            # Get activation map of an image
                            act_max = act_maxs_all_layers[layer][i]

                            # Remove outlier / non-semantic channels
                            # Set their activation maximum as 0, 
                            # so that they can be ignored later
                            if layer == 'mixed3a':
                                act_max[67] = 0
                                act_max[190] = 0

                            if layer == 'mixed3b':
                                act_max[390] = 0
                                act_max[399] = 0
                                act_max[412] = 0

                            # Sort the neurons by the activation strength
                            normalized_act_max = act_max / np.sum(act_max)
                            sorted_normalized_act_max, sorted_idxs = (list(t) for t in 
                                zip(*sorted(zip(normalized_act_max, list(range(normalized_act_max.shape[0]))), reverse=True)))

                            # Get highly activated neurons
                            prob_mass, k = 0, 0
                            while prob_mass < prob_mass_threshold:
                                prob_mass += sorted_normalized_act_max[k]
                                k += 1
                            top_neurons = sorted_idxs[:k]

                            # Add the top neurons into the result
                            for top_neuron in top_neurons:
                                As[layer][labels[i] - 1][top_neuron] += 1

                        pbar.update(len(labels))

        except tf.errors.OutOfRangeError:
            pass

# Time checker
end_time = time()
print('Total time: %.2lf sec' % (end_time - start_time))

## Save A-matrices

In [19]:
for layer in layer_sizes:
    output_filepath = '{}/A-{}-{}.csv'.format(output_dir_path, prob_mass_threshold, layer)
    np.savetxt(output_filepath, A, delimiter=',', fmt='%i')